In [ ]:
# Cell 1: Imports
import sys
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

sys.path.append(os.path.abspath(os.path.join('..')))
from src.classification_library import load_data, clean_data

# Cell 2: Parameters
# tags=["parameters"]
RAW_ZIP_PATH = '../data/raw/PRSA2017_Data_20130301-20170228.zip'
OUTPUT_SEMI_PATH = '../data/processed/dataset_semi_supervised.parquet'
LABEL_RATIO = 0.1  # Chỉ giữ lại 10% dữ liệu có nhãn, 90% sẽ gán nhãn -1 (unlabeled)
SEED = 42

# Cell 3: Main Execution
print("--- BẮT ĐẦU: CHUẨN BỊ DỮ LIỆU SEMI-SUPERVISED ---")

# 1. Load Data
df = clean_data(load_data(RAW_ZIP_PATH))

# 2. Tạo nhãn AQI (Classification Task)
# Quy ước đơn giản: 0=Good (<35), 1=Moderate (35-75), 2=Bad (>75)
# Lưu ý: Đây là cách chia bin đơn giản để demo
def create_aqi_label(pm25):
    if pm25 <= 35: return 0
    elif pm25 <= 75: return 1
    else: return 2

print("Đang tạo nhãn AQI...")
# Loại bỏ dòng thiếu PM2.5 trước khi tạo nhãn
df = df.dropna(subset=['PM2.5'])
df['AQI_Label'] = df['PM2.5'].apply(create_aqi_label)

# 3. Chọn Features (QUAN TRỌNG: Phải bỏ PM2.5 ra khỏi features để tránh Data Leakage)
feature_cols = ['PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM']
X = df[feature_cols]
y = df['AQI_Label']

# 4. Chia Train/Test (Giữ lại Test set sạch hoàn toàn để đánh giá)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED, shuffle=False)

# 5. Tạo Masking cho Train Set (Giả lập thiếu nhãn)
# Copy y_train ra để xử lý
y_train_semi = y_train.copy().values

# Tính số lượng mẫu sẽ giữ nhãn
n_labeled = int(len(y_train) * LABEL_RATIO)
indices = np.arange(len(y_train))
np.random.shuffle(indices)

# Các index không thuộc nhóm n_labeled sẽ bị gán nhãn -1 (quy ước của sklearn cho unlabeled data)
unlabeled_indices = indices[n_labeled:]
y_train_semi[unlabeled_indices] = -1

print(f"Tổng số mẫu Train: {len(y_train)}")
print(f"Số mẫu có nhãn (Labeled): {n_labeled} ({LABEL_RATIO*100}%)")
print(f"Số mẫu mất nhãn (Unlabeled): {len(unlabeled_indices)}")

# 6. Đóng gói và lưu
# Lưu X_train, y_train_semi (có -1), X_test, y_test (nhãn gốc)
data_dict = {
    "X_train": X_train,
    "y_train_semi": pd.Series(y_train_semi, index=X_train.index), # label train (có -1)
    "y_train_true": y_train, # label train gốc (để tham chiếu nếu cần)
    "X_test": X_test,
    "y_test": y_test
}

# Dùng joblib để lưu dictionary các dataframe cho tiện
import joblib
os.makedirs(os.path.dirname(OUTPUT_SEMI_PATH), exist_ok=True)
joblib.dump(data_dict, OUTPUT_SEMI_PATH)
print(f"Đã lưu dataset bán giám sát tại: {OUTPUT_SEMI_PATH}")